In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/kaggle-survey-2021/kaggle_survey_2021_responses.csv
/kaggle/input/kaggle-survey-2021/supplementary_data/kaggle_survey_2021_methodology.pdf
/kaggle/input/kaggle-survey-2021/supplementary_data/kaggle_survey_2021_answer_choices.pdf


In [2]:
# load dataset
data = pd.read_csv("../input/kaggle-survey-2021/kaggle_survey_2021_responses.csv", low_memory = False)

In [3]:
# first five rows
data.head()

,Time from Start to Finish (seconds),Q1,Q2,Q3,Q4,Q5,Q6,Q7_Part_1,Q7_Part_2,Q7_Part_3,...,Q38_B_Part_3,Q38_B_Part_4,Q38_B_Part_5,Q38_B_Part_6,Q38_B_Part_7,Q38_B_Part_8,Q38_B_Part_9,Q38_B_Part_10,Q38_B_Part_11,Q38_B_OTHER
0,Duration (in seconds),What is your age (# years)?,What is your gender? - Selected Choice,In which country do you currently reside?,What is the highest level of formal education ...,Select the title most similar to your current ...,For how many years have you been writing code ...,What programming languages do you use on a reg...,What programming languages do you use on a reg...,What programming languages do you use on a reg...,...,"In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor..."
1,910,50-54,Man,India,Bachelor’s degree,Other,5-10 years,Python,R,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,784,50-54,Man,Indonesia,Master’s degree,Program/Project Manager,20+ years,NaN,NaN,SQL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
3,924,22-24,Man,Pakistan,Master’s degree,Software Engineer,1-3 years,Python,NaN,NaN,...,NaN,NaN,TensorBoard,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,575,45-49,Man,Mexico,Doctoral degree,Research Scientist,20+ years,Python,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN


In [4]:
# size of the dataset
data.shape # 25973 rows, 369 columns

(25974, 369)

In [5]:
def rename_columns(df):
    """
    input: the dataset we want to rename the columns
    output: combine the first row of the dataset into the original column
    """
    original_columns = df.columns
    num_col = df.shape[1]
    first_row = df.iloc[0] #grab the first row for the header
    df = df[1:] #take the data below the first row
    # create a list containing new column names
    new_cols = []
    for col in range(num_col):
        new_col_name = original_columns[col] + '_' + first_row[col]
        new_cols.append(new_col_name)
    df.columns = new_cols # assign the new column names to the dataset
    return df

In [6]:
def replace_nan(df):
    """
    input:
    df - the target dataset
    
    output:
    a new dataset with nan values replaced as 0 and non-nan values replaced with 1
    """
    array = np.where(df.isnull(),0,1)
    df = pd.DataFrame(data=array, columns=df.columns)
    
    return df


#role_data = replace_nan(role_question)

In [7]:
renamed_data = rename_columns(data)

In [8]:
demographic_data = renamed_data.iloc[: , 1:7]

In [9]:
demographic_data['participant_id'] = range(0, demographic_data.shape[0])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:
def pivot_col(df, col , aggr = 'participant_id'):
    """
    input:
    df - target dataset
    col - the column we want to pivot its value as new columns
    aggr - the column we used to group by the dataset
    
    output:
    return a pivoted dataframe where columns are value from the col of old dataframe
    """
    
    pivoted_df = df.pivot(index = aggr, columns=col, values=col).reset_index().iloc[: , 1:]
    pivoted_df.index = pivoted_df.index + 1
    return pivoted_df

In [11]:
pivot_col(demographic_data, "Q1_What is your age (# years)?")

piv = pivot_col(demographic_data, "Q1_What is your age (# years)?")

pivot_col(demographic_data, "Q1_What is your age (# years)?")


Q1_What is your age (# years)?,18-21,22-24,25-29,30-34,35-39,40-44,45-49,50-54,55-59,60-69,70+
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50-54,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50-54,NaN,NaN,NaN
3,NaN,22-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,45-49,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,45-49,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
25969,NaN,NaN,NaN,30-34,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25970,NaN,22-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25971,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50-54,NaN,NaN,NaN
25972,NaN,NaN,NaN,NaN,NaN,NaN,45-49,NaN,NaN,NaN,NaN


In [12]:
def pivot_df(df, var_cols, aggr = 'participant_id'):
    """
    input:
    df - targer dataframe
    var_cols - a list of column names we want to pivot
    aggr - the column we used to group by the dataset
        
    output:
    return a dataframe where each column comes from value of each col of old dataframe
    NaN value replaced with 0 while non-NaN value replaced with 1
    """
    
    pivoted_df = []
    for col in var_cols:
        pivoted = pivot_col(demographic_data, col, aggr)
        pivoted_df.append(pivoted)
        #pivoted_df = pd.merge(pivoted, pivoted, left_index=True, right_index=True)  
    pivoted_df = pd.concat(pivoted_df, axis=1, ignore_index=False)
    pivoted_df = replace_nan(pivoted_df)
    #pivoted_df['participant_id'] = pivoted_df.index
    pivoted_df.index = pivoted_df.index + 1
    return pivoted_df
demo_matrix = pivot_df(demographic_data, demographic_data.columns[:-1])

In [13]:
demo_matrix

,18-21,22-24,25-29,30-34,35-39,40-44,45-49,50-54,55-59,60-69,...,Software Engineer,Statistician,Student,1-3 years,10-20 years,20+ years,3-5 years,5-10 years,< 1 years,I have never written code
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
5,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25969,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
25970,0,1,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
25971,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
25972,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [14]:
#demo_matrix
participant_matrix = demo_matrix.dot(np.transpose(demo_matrix))

In [15]:
participant_matrix

,1,2,3,4,5,6,7,8,9,10,...,25964,25965,25966,25967,25968,25969,25970,25971,25972,25973
1,6,2,1,1,3,1,1,4,2,2,...,2,3,2,3,3,2,1,2,2,3
2,2,6,2,2,1,0,0,1,1,1,...,3,1,2,1,1,1,2,2,2,1
3,1,2,6,1,1,0,1,1,2,2,...,1,2,3,2,2,2,4,1,2,1
4,1,2,1,6,3,0,0,1,1,1,...,2,1,1,1,1,1,1,3,2,1
5,3,1,1,3,6,2,1,2,1,2,...,1,1,2,2,1,1,1,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25969,2,1,2,1,1,0,1,3,2,3,...,2,2,3,5,2,6,2,1,1,2
25970,1,2,4,1,1,0,2,1,2,2,...,1,3,3,2,1,2,6,1,2,1
25971,2,2,1,3,2,0,0,1,1,1,...,1,1,1,1,1,1,1,6,1,2
25972,2,2,2,2,2,0,0,3,1,1,...,2,2,2,1,2,1,2,1,6,1


In [16]:
def closest_participant(participant_id, participant_matrix):
    """
    input:
    participant_id - target participant
    participant_matrix - matrix where shows the similarity between each participant
    
    output - the list of participants other than the target participant, ranked by similarity
    """
    participant_list = participant_matrix[[participant_id]]
    participant_list = participant_list.sort_values(by = participant_id, ascending = False)
    
    return participant_list.index[1:10]

In [17]:
closest_participant(2000,participant_matrix)

Int64Index([5471, 14563, 22367, 11712, 6881, 2442, 4149, 972, 4077], dtype='int64')

In [18]:
def compensation(df , participant):
    """
    input -
    df - target dataset
    participant - the id of the participant
    
    output -
    the yearly compensation of that participant
    """
    
    compensation = df.loc[df.index == participant]['Q25_What is your current yearly compensation (approximate $USD)?'].iloc[0]
    
    return compensation

compensation(renamed_data,2000)

'40,000-49,999'

In [19]:
compensation(renamed_data,5471)

'$0-999'

In [20]:
demographic_data.loc[demographic_data.index == 2000]

,Q1_What is your age (# years)?,Q2_What is your gender? - Selected Choice,Q3_In which country do you currently reside?,Q4_What is the highest level of formal education that you have attained or plan to attain within the next 2 years?,Q5_Select the title most similar to your current role (or most recent title if retired): - Selected Choice,Q6_For how many years have you been writing code and/or programming?,participant_id
2000,30-34,Woman,Japan,Bachelor’s degree,Machine Learning Engineer,3-5 years,1999


In [21]:
demographic_data.loc[demographic_data.index == 5471]

,Q1_What is your age (# years)?,Q2_What is your gender? - Selected Choice,Q3_In which country do you currently reside?,Q4_What is the highest level of formal education that you have attained or plan to attain within the next 2 years?,Q5_Select the title most similar to your current role (or most recent title if retired): - Selected Choice,Q6_For how many years have you been writing code and/or programming?,participant_id
5471,30-34,Woman,China,Master’s degree,Machine Learning Engineer,3-5 years,5470


In [22]:
"""
def compute_correlation(df, user1, user2):
    '''
    INPUT
    user1 - int user_id
    user2 - int user_id
    df - dataset where is a matrix of user and their pivoted answer columns
    OUTPUT
    the correlation between the matching ratings between the two users
    '''
    answer_1 = list(df.loc[df.index == user1].iloc[0])
    answer_2 = list(df.loc[df.index == user2].iloc[0])
    
    dot_product = np.vdot(answer_1, answer_2)
    
    return dot_product #return the correlation
"""

"\ndef compute_correlation(df, user1, user2):\n    '''\n    INPUT\n    user1 - int user_id\n    user2 - int user_id\n    df - dataset where is a matrix of user and their pivoted answer columns\n    OUTPUT\n    the correlation between the matching ratings between the two users\n    '''\n    answer_1 = list(df.loc[df.index == user1].iloc[0])\n    answer_2 = list(df.loc[df.index == user2].iloc[0])\n    \n    dot_product = np.vdot(answer_1, answer_2)\n    \n    return dot_product #return the correlation\n"

In [23]:
def subset_data(df, col, criteria):
    """
    input:
    df: the dataset we want to subset from
    col: target columns as the filter
    criteria: value to feed the filter
    
    output:
    a new dataset which is a subset of the original one
    """
    
    new_df = df.loc[df[col] == criteria]
    
    return new_df
"""
ds_data = subset_data(renamed_data, 
           'Q5_Select the title most similar to your current role (or most recent title if retired): - Selected Choice',
           'Data Scientist')
"""

"\nds_data = subset_data(renamed_data, \n           'Q5_Select the title most similar to your current role (or most recent title if retired): - Selected Choice',\n           'Data Scientist')\n"

In [24]:
def question_columns(df, query, method = 'strict'):
    """
    input: 
    df - target dataset
    query - str, query we want to find relevant infomation in the dataset. e.g. 'Q7', or 'machine learning' 
    
    output:
    a subset of data which include the columns of the query in interest
    
    method:
    if it == strict, which means we will look for the question exactly EQUALS to the query. e.g. if we search 'age', then 'language' won't
    be taken into account in this case;
    
    if it == loose, which means we will look for the question exactly CONTAINS the query. e.g. if we search 'age', then 'language' will
    be taken into account in this case.
    """
    columns = df.columns
    question_col = []
    for col in columns:
        if method == 'strict':
            col_parts = col.lower().split() # each column name will be separated into single word tokens at first
            if query.lower() in col_parts:
                question_col.append(col)
        elif method == 'loose':
            if query.lower() in col.lower():
                question_col.append(col)
    return df[question_col]

In [25]:
question_columns(renamed_data, query = 'compensation', method = 'strict')

,Q25_What is your current yearly compensation (approximate $USD)?
1,"25,000-29,999"
2,"60,000-69,999"
3,$0-999
4,"30,000-39,999"
5,"30,000-39,999"
...,...
25969,"15,000-19,999"
25970,NaN
25971,$0-999
25972,NaN


## What questions we want to ask here?
1. How many types of data scientists based on activity they usually do?
2. Which skill each type emphasizes at?
3. Which platform/tools each type emphazies at?
4. Demographic segmentation of each type?
5. Profession segmentation of each type?

In [26]:
#feature_data = role_data.iloc[: , 1:]

In [27]:
"""
def kmeans_cluster_opt(df, init = 'k-means++', max_num_cluster = 9):
    
    input: 
    df - the dataset we want to segments into cluster
    init - the way we want to initialize the starting centroid
    max_num_cluster - the max number of cluster
    
    output:
    a visualization showing the line graph indicating the optimal number of klusters, based on inertias value
    
    num_clusters = list(range(1, max_num_cluster))
    inertias = []

    for k in num_clusters:
        model = KMeans(init=init, n_clusters=k, random_state = 42)
        model.fit(df)
        inertias.append(model.inertia_)

    
    plt.plot(num_clusters, inertias, '-o')

    plt.xlabel('number of clusters (k)')
    plt.ylabel('inertia')

    plt.show()
"""

"\ndef kmeans_cluster_opt(df, init = 'k-means++', max_num_cluster = 9):\n    \n    input: \n    df - the dataset we want to segments into cluster\n    init - the way we want to initialize the starting centroid\n    max_num_cluster - the max number of cluster\n    \n    output:\n    a visualization showing the line graph indicating the optimal number of klusters, based on inertias value\n    \n    num_clusters = list(range(1, max_num_cluster))\n    inertias = []\n\n    for k in num_clusters:\n        model = KMeans(init=init, n_clusters=k, random_state = 42)\n        model.fit(df)\n        inertias.append(model.inertia_)\n\n    \n    plt.plot(num_clusters, inertias, '-o')\n\n    plt.xlabel('number of clusters (k)')\n    plt.ylabel('inertia')\n\n    plt.show()\n"

In [28]:
#kmeans_cluster_opt(feature_data)

In [29]:
"""
def kmeans_predict(df, init = 'k-means++', n_clusters = 4):
    
    input:
    df - dataset we want to segment into clusters
    init - the way we want to initialize the starting centroid
    n_clusters - the number of cluster
    
    output:
    labels - return an array of predictions on the cluster label of given features
    centers - centroid values of each cluster
    model = KMeans(init=init, n_clusters = n_clusters, random_state = 42)

    model.fit(df)

    labels = model.predict(df)
    
    centers = np.array(model.cluster_centers_)
    
    return labels, centers
"""

"\ndef kmeans_predict(df, init = 'k-means++', n_clusters = 4):\n    \n    input:\n    df - dataset we want to segment into clusters\n    init - the way we want to initialize the starting centroid\n    n_clusters - the number of cluster\n    \n    output:\n    labels - return an array of predictions on the cluster label of given features\n    centers - centroid values of each cluster\n    model = KMeans(init=init, n_clusters = n_clusters, random_state = 42)\n\n    model.fit(df)\n\n    labels = model.predict(df)\n    \n    centers = np.array(model.cluster_centers_)\n    \n    return labels, centers\n"